# Notebook Instructions
<i>You can run the notebook document sequentially (one cell a time) by pressing <b> shift + enter</b>. While a cell is running, a [*] will display on the left. When it has been run, a number will display indicating the order in which it was run in the notebook [8].</i>

<i>Enter edit mode by pressing <b>`Enter`</b> or using the mouse to click on a cell's editor area. Edit mode is indicated by a green cell border and a prompt showing in the editor area.</i>

## Options Trading Strategy
Strategy: Short straddle before earnings announcement and square off after that.

### Define instrument name, earnings date, and strike difference

In [1]:
instrument_name = 'INFY'

strike_difference = 20

from datetime import date, timedelta
earnings_date = date(2018,1,12)

### Set start and end date

In [2]:
start_date = earnings_date - timedelta(days=0)
end_date = earnings_date + timedelta(days=3)

print (start_date)
print (end_date)

2018-01-12
2018-01-15


### Define ATM strike price

In [3]:
from data import get_stock_data
import numpy as np
close_price = get_stock_data(instrument_name,start_date,start_date).Close.values
atm_strike_price = np.round(close_price/strike_difference,0)[0]*strike_difference
atm_strike_price

1080.0

### Set expiry date 

In [4]:
from nsepy.derivatives import get_expiry_date
expiry_date = get_expiry_date(year=end_date.year, month=end_date.month)
expiry_date

building dictionary


datetime.date(2018, 1, 25)

### Fetch Data

In [5]:
from data import get_option_data
data_ce = get_option_data(instrument_name,start_date,end_date,'CE',atm_strike_price,expiry_date)
data_ce

,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,
2018-01-12,INFY,2018-01-25,CE,1080.0,26.85,30.45,22.0,24.2,23.0,24.2,3872,2.567442e+09,58386000.0,819000,149400,1078.45
2018-01-15,INFY,2018-01-25,CE,1080.0,22.10,25.35,12.7,15.8,16.2,15.8,5334,3.513153e+09,56721000.0,706200,-112800,1081.15


In [6]:
data_pe = get_option_data(instrument_name,start_date,end_date,'PE',atm_strike_price,expiry_date)
data_pe

,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,
2018-01-12,INFY,2018-01-25,PE,1080.0,27.55,34.30,24.6,26.45,25.0,26.45,1907,1.268988e+09,33252000.0,534600,271800,1078.45
2018-01-15,INFY,2018-01-25,PE,1080.0,21.25,28.55,13.5,17.00,16.0,17.00,3632,2.392789e+09,39253000.0,515400,-19200,1081.15


### Calculate implied volatility

In [7]:
import mibian
days_to_expiry = (expiry_date - start_date).days
Call_IV = mibian.BS([close_price, atm_strike_price, 0.05, days_to_expiry], callPrice=data_ce.Close.iloc[0]).impliedVolatility
Call_IV

30.76171875

In [8]:
Put_IV = mibian.BS([close_price, atm_strike_price, 0.05, days_to_expiry], putPrice=data_pe.Close.iloc[0]).impliedVolatility
Put_IV

31.6009521484375

### Alpha

In [9]:
if Call_IV > 30 and Put_IV > 30:
    # Short Straddle
    data_pe['signal'] = -1
    data_ce['signal'] = -1
else:
    # No Positions
    data_pe['signal'] = 0
    data_ce['signal'] = 0

## Analyzing The Performance

### Compute daily returns

In [10]:
from performance_analytics import compute_ret
data_ce = compute_ret(data_ce)
data_pe = compute_ret(data_pe)
data =  (data_ce.strategy_ret + data_pe.strategy_ret)/2
data*100

Date
2018-01-12          NaN
2018-01-15    35.219266
Name: strategy_ret, dtype: float64